<a href="https://colab.research.google.com/github/yvonneleoo/Real-Time-Voice-Swapping/blob/master/waveglow_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Clone git repo
!git clone https://github.com/thegreatwarlo/Real-Time-Voice-Swapping.git

Cloning into 'Real-Time-Voice-Swapping'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 2474 (delta 14), reused 14 (delta 6), pack-reused 2441
Receiving objects: 100% (2474/2474), 364.03 MiB | 29.89 MiB/s, done.
Resolving deltas: 100% (1335/1335), done.


In [2]:
cd Real-Time-Voice-Swapping/

/content/Real-Time-Voice-Swapping


In [3]:
!pip install -q -r requirements.txt
!apt-get install -qq libportaudio2
!python -m pip install SpeechRecognition
!pip install wavio # convert wav format for speech recognition

     |████████████████████████████████| 109.2MB 109kB/s 
     |████████████████████████████████| 377.0MB 44kB/s 
     |████████████████████████████████| 686kB 49.7MB/s 
     |████████████████████████████████| 71kB 7.5MB/s 
     |████████████████████████████████| 245kB 46.9MB/s 
     |████████████████████████████████| 62.2MB 131kB/s 
     |████████████████████████████████| 32.8MB 1.5MB/s 
     |████████████████████████████████| 491kB 53.1MB/s 
     |████████████████████████████████| 3.2MB 48.9MB/s 
     |████████████████████████████████| 204kB 48.6MB/s 
     |████████████████████████████████| 266kB 40.8MB/s 
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 145605 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-1_amd64.deb ...
Unpacking libportaudio2:amd64 (19.6.0-1) ...
Setting up libportaudio2:amd64 (19.6.0-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...


In [4]:
# Download dataset
!gdown https://drive.google.com/uc?id=1n1sPXvT34yXFLT47QZA6FIRGrwMeSsZc
!unzip pretrained.zip

Downloading...
From: https://drive.google.com/uc?id=1n1sPXvT34yXFLT47QZA6FIRGrwMeSsZc
To: /content/Real-Time-Voice-Swapping/pretrained.zip
384MB [00:05, 73.3MB/s]
Archive:  pretrained.zip
   creating: encoder/saved_models/
  inflating: encoder/saved_models/pretrained.pt  
   creating: synthesizer/saved_models/
   creating: synthesizer/saved_models/logs-pretrained/
   creating: synthesizer/saved_models/logs-pretrained/taco_pretrained/
 extracting: synthesizer/saved_models/logs-pretrained/taco_pretrained/checkpoint  
  inflating: synthesizer/saved_models/logs-pretrained/taco_pretrained/tacotron_model.ckpt-278000.data-00000-of-00001  
  inflating: synthesizer/saved_models/logs-pretrained/taco_pretrained/tacotron_model.ckpt-278000.index  
  inflating: synthesizer/saved_models/logs-pretrained/taco_pretrained/tacotron_model.ckpt-278000.meta  
   creating: vocoder/saved_models/
   creating: vocoder/saved_models/pretrained/
  inflating: vocoder/saved_models/pretrained/pretrained.pt  


In [0]:
# Code for recording audio from the browser
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode
import IPython
import uuid
from google.colab import output
import speech_recognition

class InvokeButton(object):
  def __init__(self, title, callback):
    self._title = title
    self._callback = callback

  def _repr_html_(self):
    from google.colab import output
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)

    template = """<button id="{callback_id}" style="cursor:pointer;background-color:#EEEEEE;border-color:#E0E0E0;padding:5px 15px;font-size:14px">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id)
    return html

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=3, opath='audio.wav'):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  with open(opath,'wb+') as f:
    f.write(b)
  return opath

In [13]:
from IPython.display import Audio
from IPython.utils import io
from synthesizer.inference import Synthesizer
from encoder import inference as encoder
from vocoder import inference as vocoder
from pathlib import Path
import numpy as np
import librosa
import os
from scipy.io import wavfile
import wavio
from scipy.io.wavfile import write

from encoder.params_model import model_embedding_size as speaker_embedding_size
from utils.argutils import print_args
import argparse
import torch
import sys
import speech_recognition
import time

encoder_weights = Path("encoder/saved_models/pretrained.pt")
vocoder_weights = Path("vocoder/saved_models/pretrained/pretrained.pt")
syn_dir = Path("synthesizer/saved_models/logs-pretrained/taco_pretrained")
encoder.load_model(encoder_weights)
synthesizer = Synthesizer(syn_dir)
vocoder.load_model(vocoder_weights)

Loaded encoder "pretrained.pt" trained to step 1564501
Found synthesizer "pretrained" trained to step 278000
Building Wave-RNN
Trainable Parameters: 4.481M
Loading model weights at vocoder/saved_models/pretrained/pretrained.pt


In [16]:
def clone_voice(in_fpath_1, in_fpath_2, encoder, synthesizer, num_generated, waveglow):
            
            # play the original voices
            if num_generated == 0:
                print('The first voice reference(A):')
                display(Audio(in_fpath_1))
                print('The second voice reference(B):')
                display(Audio(in_fpath_2))

            # load files and preprocessing
            start_time = time.time()            
            preprocessed_wav = encoder.preprocess_wav(Path(in_fpath_1))
            original_wav, sampling_rate = librosa.load(Path(in_fpath_1))               
            preprocessed_wav = encoder.preprocess_wav(original_wav, sampling_rate)      
            d = round(time.time() - start_time, 4)
            print(f"Loaded file and preprocessed succesfully in {d} seconds")                        
            
            # embedding
            start_time = time.time()
            embed = encoder.embed_utterance(preprocessed_wav)     
            d = round(time.time() - start_time, 4)
            print(f"Created the embedding in {d} seconds")
            
            ##Generating text from other speaker's speech
            start_time = time.time()
            wav, rate = librosa.load(Path(in_fpath_2))   
            y = (np.iinfo(np.int32).max * (wav/np.abs(wav).max())).astype(np.int32) # Convert `data` to 32 bit integers:
            wavfile.write(in_fpath_2, rate, y)
            recognizer = speech_recognition.Recognizer()    
            with speech_recognition.AudioFile(in_fpath_2) as source:
                audio = recognizer.record(source) 
            text = recognizer.recognize_google(audio)
            texts = [text]
            embeds = [embed]

            
            ## Generating the spectrogram
            start_time = time.time()      
            d = round(time.time() - start_time, 4)
            print(f"Loaded file and Extracted text from speech in {d} seconds")    
            start_time = time.time()
            specs = synthesizer.synthesize_spectrograms(texts, embeds)
            spec = specs[0]
            d = round(time.time() - start_time, 4)
            print(f"Created the mel spectrogram in {d} seconds")            
            
            ## Generating the waveform
            print("Synthesizing the waveform:")
            start_time = time.time()
            waveglow = waveglow.remove_weightnorm(waveglow)
            waveglow = waveglow.to('cuda:0', dtype=torch.float64)
            waveglow.eval()
            
            with torch.no_grad():
                spec = torch.from_numpy(spec).to('cuda:0', dtype=torch.float64) # convert to tensor
                spec = spec.unsqueeze(0)
                audio = waveglow.infer(spec)
            audio_numpy = audio[0].data.cpu().numpy()
            d = round(time.time() - start_time, 4)
            print(f"\nGenerated the waveform in {d} seconds")
            
            # Play the audio (non-blocking)
            if num_generated == 0:
              print("A's voice + B's text")
              display(Audio(audio_numpy, rate=rate))
            else:
              print("B's voice + A's text")
              display(Audio(audio_numpy, rate=rate))
                                
            # Save it on the disk
            fpath = "demo_output_%02d.wav" % num_generated
            write(fpath, rate, audio_numpy)
            
            print("\nSaved output as %s\n\n" % fpath)

if __name__ == '__main__':
  print("Interactive generation loop")
  num_generated = 0

  # record the reference audio and get the filepath
  start_time = time.time()
  print("Now recording the first voice reference for 8 seconds, and less than 20 words.")
  record(sec=8, opath='original_voice_A.wav')
  print("Audio recording and complie complete")
  in_fpath_1 = 'original_voice_A.wav'

  print("Now recording the second voice reference for 8 seconds, and less than 20 words.")
  record(sec=8, opath='original_voice_B.wav')
  print("Audio recording and compile complete")
  in_fpath_2 = 'original_voice_B.wav'

  d = round(time.time() - start_time, 4)
  print(f"\nRecording in {d} seconds")
  
  #first speaker's voice, second speaker's words
  waveglow = torch.hub.load('nvidia/DeepLearningExamples:torchhub', 'nvidia_waveglow')
  start_time = time.time()
  clone_voice(in_fpath_1, in_fpath_2, encoder, synthesizer, num_generated, waveglow = waveglow)
  duration = round(time.time() - start_time, 4)
  print(f"first speaker's voice applied to second speaker's words in {duration} seconds")
  num_generated += 1  

  waveglow = torch.hub.load('nvidia/DeepLearningExamples:torchhub', 'nvidia_waveglow')  
  start_time = time.time()
  clone_voice(in_fpath_2, in_fpath_1, encoder, synthesizer, num_generated, waveglow = waveglow)
  duration = round(time.time() - start_time, 4)
  print(f"second speaker's voice applied to first speaker's words in {duration} seconds")
  num_generated += 1

Interactive generation loop
Now recording the first voice reference for 8 seconds, and less than 20 words.


<IPython.core.display.Javascript object>

Audio recording and complie complete
Now recording the second voice reference for 8 seconds, and less than 20 words.


<IPython.core.display.Javascript object>

Audio recording and compile complete

Recording in 19.7411 seconds


...

Loaded file and preprocessed succesfully in 3.2043 seconds
Created the embedding in 0.0427 seconds
Loaded file and Extracted text from speech in 2.8663 seconds
Created the mel spectrogram in 0.6118 seconds
Synthesizing the waveform:

Generated the waveform in 13.6857 seconds
A's voice + B's text



Saved output as demo_output_00.wav


first speaker's voice applied to second speaker's words in 22.0128 seconds


Using cache found in /root/.cache/torch/hub/nvidia_DeepLearningExamples_torchhub


Loaded file and preprocessed succesfully in 2.0765 seconds
Created the embedding in 0.0417 seconds
Loaded file and Extracted text from speech in 2.7638 seconds
Created the mel spectrogram in 0.5631 seconds
Synthesizing the waveform:

Generated the waveform in 13.3685 seconds
B's voice + A's text



Saved output as demo_output_01.wav


second speaker's voice applied to first speaker's words in 20.3885 seconds
